# Vorverarbeitung der Reden für das Training eines LLM

In diesem Skript werden einige Vorverarbeitungsschritte gemacht, um die Daten für das Training eines LLMs (z.B. Gottbert) vorzubereiten.
Konkret werden in diesem Skript An- und Abreden entfernt, und im Anschluss werden die Reden in kleinere Abschnitte unterteilt. Dann wird ein Datensatz erstellt, in dem Parteinamen entfernt

Zum Schluss erfolgt die Aufteilung in Trainings-, Validierungs- und Testdaten, sowie die Augmentierung der Trainingsdaten um zu berücksichtigen, dass manche Parteien weniger stark repräsentiert sind.

## Vorbereitungen

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import random
import math
from datetime import datetime
import pickle

## Daten laden und Grundlegendes

In [ ]:
path = '/content/drive/MyDrive/techlabs/Github/Daten/'
df = pd.read_parquet(path + 'datatechlabs_2020_newclean_cleantext.parquet')

Numerische Labels erstellen:

In [ ]:
df['labels'] = df['Fraktion']
fraktionen = df['labels'].unique()
mapping = {fraktion: i for i, fraktion in enumerate(fraktionen)}
df['labels'] = df['labels'].map(mapping)

Mapping speichern (damit man es nach dem Training für die Performance Evaluation laden kann):

In [ ]:
with open(path + 'party_mapping.pkl', 'wb') as fp:
    pickle.dump(mapping, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file


Ein paar grundlegende Informationen anzeigen lassen:

In [ ]:
print(df['Fraktion'].value_counts())
print(df.head())

Fraktion
CDU/CSU       10083
SPD            9287
Die Grünen     6221
FDP            5213
AfD            4824
Die Linke      3263
Name: count, dtype: int64
  Wahlperiode       Datum Redner*in-ID Redner*in Vorname Redner*in Nachname  \
0          19  15.01.2020     11004699            Astrid            Damerow   
1          19  15.01.2020     11004393            Johann           Saathoff   
2          19  15.01.2020     11003706             Artur        Auernhammer   
3          19  15.01.2020     11003604         Friedrich         Ostendorff   
4          19  15.01.2020     11003740           Heidrun      Bluhm-Förster   

   Rolle    Fraktion                                               Rede  \
0  keine     CDU/CSU  Frau Präsidentin! Verehrte Kolleginnen und Kol...   
1  keine         SPD  Sehr geehrte Frau Präsidentin! Liebe Kolleginn...   
2  keine     CDU/CSU  Verehrte Frau Präsidentin! Liebe Kolleginnen u...   
3  keine  Die Grünen  Sehr geehrte Frau Präsidentin! Liebe Kolleginn..

In [ ]:
print(df.iloc[0]['Rede'])

Frau Präsidentin! Verehrte Kolleginnen und Kollegen! Die Debatte hat es gezeigt: Niemand hier im Haus wird bezweifeln, dass Wasser eine Ressource von wirklich unschätzbarem Wert ist. Der Antrag der Grünen, der hier noch nicht so sehr besprochen wurde, hat die Überschrift: "Wasser muss sauber und bezahlbar bleiben". Ich will an dieser Stelle nochmals wie übrigens bereits im Mai deutlich machen, dass wir das Ziel und das Anliegen der Antragsteller hier absolut teilen. Was wir allerdings nicht teilen, sind die Bewertungen und die Schlussfolgerungen, die Sie in Ihrem Antrag ziehen. Ich widerspreche hier nochmals ausdrücklich dem immer wieder erweckten Eindruck, dass wir bei der Wasserqualität in den vergangenen Jahren keine Fortschritte erzielt haben. Ganz im Gegenteil: Überall in Deutschland kann Wasser aus der Leitung getrunken werden. Das ist weiß Gott keine Selbstverständlichkeit. Verehrte Kolleginnen und Kollegen, Kernstück unseres Gewässerschutzes ist die europäische Wasserrahmenrich

Wir behalten nur Reden, die eine bestimmte Mindestlänge haben (hier: 200 Zeichen):

In [ ]:
df['AnzahlZeichen'] = df['Rede'].apply(len).astype("int16")
df['AnzahlWoerter'] = df['Rede'].apply(lambda x: len(x.split())).astype("int16")

df = df[df['AnzahlWoerter'] >= 200]
df = df[df['AnzahlZeichen']  >= 0]

## Anreden rausnehmen

Hier nutzen wir einen recht groben Ansatz, um die Anreden (und 'Abreden') der Reden zu entfernen: Wir nehmen vorne die ersten beiden Sätze und hinten den letzten Satz weg.

In [ ]:
def strip_speech(speech):

  # find first split point:
  first_fs = speech.find(".") # first full stop
  first_em = speech.find("!") # first exclamation mark

  if first_em == -1:
    first_em = len(speech)+10 # if there is no exclamation mark, set this to a large number so -1 is not considered as a split point

  if first_fs < first_em:
    first_sp = first_fs
  else:
    first_sp = first_em

  second_fs = speech.find(".", first_sp+1) # second full stop
  second_em = speech.find("!", first_sp+1) # second exclamation mark

  if second_em == -1:
    second_em = len(speech)+10 # if there is no exclamation mark, set this to a large number so -1 is not considered as a split point

  if second_fs < second_em:
    second_sp = second_fs
  else:
    second_sp = second_em

  # find last split point:
  last_fs = speech.rfind(".")
  last_em = speech.rfind("!")

  if last_fs > last_em:
    last_sp = last_fs
  else:
    last_sp = last_em

  ntlast_fs = speech.rfind(".",0,last_sp)
  ntlast_em = speech.rfind("!",0,last_sp)

  if ntlast_fs > ntlast_em:
    ntlast_sp = ntlast_fs
  else:
    ntlast_sp = ntlast_em

  # split speech at both points (retain everything in the middle):
  speech = speech[(second_sp+2):(ntlast_sp+1)]

  return speech

Testen:

In [ ]:
speech = df.iloc[15]['Rede']
stripped_speech = strip_speech(speech)

print(speech)
print(stripped_speech)

print(speech[(len(speech)-100):])
print(stripped_speech[(len(stripped_speech)-100):])

Sehr geehrte Frau Präsidentin! Liebe Kolleginnen und Kollegen! Auch wir Freie Demokraten sind selbstverständlich für ein höchstmögliches Maß an Sicherheit im Luftverkehr. Wir finden es auch richtig wie es in dem Vorschlag beschrieben wird und allgemein im Bereich der Luftsicherheit gilt , dass Unternehmen einschließlich Luftfahrtunternehmen, Betreiber von Flughäfen, Dienstleister an Flughäfen, Verkehrspiloten und Berufspiloten luftsicherheitsrechtlichen Zuverlässigkeitsüberprüfungen unterzogen werden. Aber mit einer Zuverlässigkeitsüberprüfung ohne Differenzierung zwischen der Reinigungskraft und dem Verkehrspiloten in jedem Ort, im Flughafen oder auf einem kleinen Segelfluggelände denn der Motorsegler ist auch davon betroffen; das gilt auch für den Segelflieger, der eine Motorsegelflugberechtigung hat , wird alles über einen Kamm geschert. Selbstverständlich macht das einen Riesenunterschied. Sie machen aber weiterhin dabei keinen Unterschied, und das ist nicht in Ordnung. Ich kann es

Und auf den gesamten Datensatz anwenden:

In [ ]:
df['Rede_bereinigt'] = df['Rede'].apply(lambda s: strip_speech(s))

Wir berechnen erneut die Anzahl der Zeichen und Wörter, und nehmen die Reden raus, die zu kurz sind:

In [ ]:
df['AnzahlZeichen_b'] = df['Rede_bereinigt'].apply(len).astype("int16")
df['AnzahlWoerter_b'] = df['Rede_bereinigt'].apply(lambda x: len(x.split())).astype("int16")

In [ ]:
df = df[df['AnzahlWoerter_b'] >= 200]
df = df[df['AnzahlZeichen_b']  >= 0]

## Rede in kleinere Abschnitte aufteilen

Hier teilen wir die Rede in Abschnitte auf, die von der Länge her ungefähr der maximalen Anzahl an Tokens entsprechen, mit denen das Modell umgehen kann (512). Wir definieren dafür eine Funktion:

In [ ]:
def divide_speech(speech, string_size, word_tolerance):

 parts = math.ceil(len(speech)/string_size)

 if parts == 1: # wenn es nur einen Abschnitt gibt, nehme die ganze speech
  return [speech]
 else:
  order = ["begin", "end"]
  this_order = random.sample(order, 1)


  divided_speech = []

  for i in range(0, parts):

    if this_order[0] == "begin":
      part = speech[(i*string_size):(i+1)*string_size]

    if this_order[0] == "end":
      if (len(speech)-(i+1)*string_size) < 0:
        part = speech[0 :(len(speech)-i*string_size)]
      else:
        part = speech[(len(speech)-(i+1)*string_size) :(len(speech)-i*string_size)]

    #Sample, wenn möglich, ganze Sätze (beginne und ende nach diesen)
    start = part.find(".") + 2
    end = part.rfind(".") + 1
    if (i == 0 and this_order[0] == "begin") or (i == (parts - 1) and this_order[0] == "end"):
        start = 0
    if (i == 0 and this_order[0] == "end") or (i == (parts - 1) and this_order[0] == "begin"):
        end = len(part)
    part = part[start:end]

    # Ermittle, ob das Sample die word_tolerance erfüllt (sonst verwirf es)
    word_number = len(part.split())


    if word_number >= word_tolerance:
      divided_speech.append(part)

  # zufällige Reihenfolge der samples:
  random.shuffle(divided_speech)
  return divided_speech

Dann überprüfen wir, ob sie funktioniert:

In [ ]:
speech = df.iloc[10]['Rede_bereinigt']
print(speech)

divided_speech = divide_speech(speech, string_size = 3000, word_tolerance = 200)

print(divided_speech)
print(len(divided_speech))

Luftverkehr ist besonders verletzlich das ist mehrfach schon erwähnt worden und unterliegt daher einer besonderen Gefährdung. Innentätern das ist auch schon erwähnt worden , also Menschen, die besonderen Zugang zu Flughäfen und ihren Einrichtungen haben, kommt dabei eine zentrale Rolle zu. Denn: Wer kann besser sabotieren oder manipulieren als jemand, der sich richtig gut auskennt? Und wer ist dadurch für Attentäter ein besonders attraktiver Komplize? Er muss noch nicht mal selber der Attentäter sein. Wir reden hier aber nicht nur von der Gefahr terroristischer Anschläge, sondern auch von organisierter Kriminalität, die Kriminelle einschleust. Die Frage ist also, wie wir uns am besten vor Innentätern schützen können und dies, ohne dass die ganze Branche unter den immer wieder erwähnten Generalverdacht gerät. Der vorliegende Gesetzentwurf liefert mit der neuen Zuverlässigkeitsüberprüfung für Menschen, die in der Luftsicherheit arbeiten, erste überzeugende Antworten. Wir müssen doch wiss

Und schließlich wenden wir sie auf unseren Datensatz an:

In [ ]:
df_snippets = pd.DataFrame()

for s in range(0, len(df)):
  split_speech = divide_speech(df.iloc[s]['Rede_bereinigt'], string_size = 3000, word_tolerance = 200)
  if len(split_speech) > 0:
    df_row = pd.DataFrame(df.iloc[s])
    df_rows = pd.concat([df_row.T]*len(split_speech))
    df_rows['Rede_bereinigt'] = split_speech
    df_snippets = pd.concat([df_snippets, df_rows])

Wir zählen erneut Zeichen und Wörter und schauen uns die Verteilung pro Partei an:

In [ ]:
df_snippets['AnzahlZeichen_s'] = df_snippets['Rede_bereinigt'].apply(len).astype("int16")
df_snippets['AnzahlWoerter_s'] = df_snippets['Rede_bereinigt'].apply(lambda x: len(x.split())).astype("int16")

In [ ]:
df_snippets.groupby('Fraktion')['AnzahlZeichen_s'].describe()

,count,mean,std,min,25%,50%,75%,max
Fraktion,,,,,,,,
AfD,4377.0,2684.650217,420.243166,1256.0,2620.00,2879.0,2948.0,3000.0
CDU/CSU,11121.0,2620.811528,470.479934,1250.0,2445.00,2861.0,2943.0,3000.0
Die Grünen,5676.0,2654.440980,427.039491,1245.0,2543.75,2857.0,2940.0,3000.0
Die Linke,2917.0,2613.284539,435.542327,1237.0,2407.00,2823.0,2935.0,3000.0
FDP,4947.0,2626.189812,467.671373,1260.0,2442.50,2869.0,2944.5,3000.0
SPD,10044.0,2648.949721,459.395727,1224.0,2556.75,2874.0,2947.0,3000.0


In [ ]:
df_snippets.groupby('Fraktion')['AnzahlZeichen_s'].describe()
#Verteilungen sind nicht mehr ganz so ähnlich wie zuvor (allerdings später beim train_df wieder okay.)

,count,mean,std,min,25%,50%,75%,max
Fraktion,,,,,,,,
AfD,4373.0,2686.189115,419.380774,1256.0,2623.0,2881.0,2948.0,3000.0
CDU/CSU,11098.0,2624.176879,467.323096,1238.0,2456.0,2862.0,2944.0,3000.0
Die Grünen,5678.0,2654.787777,427.892293,1245.0,2540.0,2860.0,2941.0,3000.0
Die Linke,2912.0,2615.480769,434.170073,1237.0,2411.0,2825.0,2935.0,3000.0
FDP,4946.0,2629.742620,467.013851,1260.0,2449.0,2871.0,2948.0,3000.0
SPD,10049.0,2648.572097,459.690230,1224.0,2558.0,2873.0,2947.0,3000.0


In [ ]:
#df_snippets.groupby('Fraktion')['AnzahlWoerter_s'].describe()

,count,mean,std,min,25%,50%,75%,max
Fraktion,,,,,,,,
AfD,5862.0,261.645855,23.187259,200.0,248.0,265.0,278.0,326.0
CDU/CSU,14539.0,266.598322,23.518286,200.0,253.0,270.0,283.0,344.0
Die Grünen,7172.0,266.520357,23.769943,200.0,253.0,270.0,283.0,328.0
Die Linke,3477.0,266.351452,22.612794,200.0,254.0,269.0,282.0,328.0
FDP,6387.0,266.030844,23.768210,200.0,253.0,269.0,283.0,334.0
SPD,13365.0,265.863823,23.453435,200.0,252.0,269.0,282.0,335.0


Die Verteilungen sind jetzt auf jeden Fall sehr ähnlich.

In [ ]:
df_snippets.groupby('Fraktion')['AnzahlWoerter_s'].describe()
#von den Worten wiederum in Ordnung

,count,mean,std,min,25%,50%,75%,max
Fraktion,,,,,,,,
AfD,4377.0,379.742518,60.511011,200.0,363.0,399.0,419.0,478.0
CDU/CSU,11121.0,378.938045,69.251736,200.0,350.0,405.0,427.0,509.0
Die Grünen,5676.0,382.911381,63.161130,200.0,360.0,404.0,426.0,489.0
Die Linke,2917.0,373.110045,64.423101,200.0,338.0,394.0,420.0,489.0
FDP,4947.0,378.202143,68.578239,200.0,350.0,403.0,426.0,514.0
SPD,10044.0,382.098467,67.505975,200.0,363.0,406.0,427.0,495.0


## Parteinamen rausnehmen

Damit das Modell später nicht anhand der Häufigkeit bestimmter Parteinamen lernt, welcher Partei eine Rede zuzuordnen ist, versuchen wir, alle Parteinamen zu entfernen. Dazu erstellen wir zunächst eine Funktion, die in den Reden Wörter aus einer Liste durch ein anderes Wort ersetzen kann:

In [ ]:
def remove_substrings_replace(speech, substrings, replacement):
    for substring in substrings:
        speech = speech.replace(substring, replacement)
    return speech

Dann definieren wir die Liste mit den zu ersetzenden Parteinamen

In [ ]:
substrings = ["FDP", "Freie Demokratische Partei",  "Freien Demokratischen Partei", "Freie-Demokratische Partei",  "Freien-Demokratischen Partei",
              "Freien Demokraten", "Freie Demokraten",  "Freie Demokratin", "Freier Demokrat", "Liberalen", "Liberaler", "Liberale",

              "SPD", "Sozialdemokratische Partei Deutschlands", "Sozialdemokratischen Partei Deutschlands",
              "Sozialdemokratische Partei", "Sozialdemokratischen Partei", "Sozial-Demokratische Partei", "Sozial-Demokratischen Partei",
               "Sozial-demokratische Partei", "Sozial-demokratischen Partei", "Sozialdemokraten", "Sozialdemokratin", "Sozialdemokrat",

              "CDU/CSU",
              "CDU", "Christlich Demokratische Union", "Christlich Demokratischen Union", "Christlich-Demokratische Union",
              "Christlich-Demokratischen Union", "Christlich-demokratische Union", "Christlich-demokratischen Union",

              "CSU", "Christlich-Soziale Union", "Christlich-Sozialen Union", "Christlich Soziale Union", "Christlich Sozialen Union",
              "Christlich-soziale Union", "Christlich-sozialen Union", "Union",
              "Christdemokraten" ,  "Christdemokratin" , "Christdemokrat",

              "Bündnis 90/Die Grünen", "Bündnis 90", "Grünen", "Grüner", "Grüne",

              "Linken", "Linker", "Linke",

              "AFD", "AfD",  "Alternativen für Deutschland", "Alternative für Deutschland"]

Wir testen die geschriebene Funktion an einem Testobjekt:

In [ ]:
test = remove_substrings_replace("Wir, die Grünen, vom Bündnis 90/Die Grünen, lehnen natürlich jegliche Linke links von der AFD ab, und die liberalen Menschen von der Freien Demokratischen Partei auch, dafür sind wir freie Demokraten!",
                                 substrings = substrings, replacement = "Partei")
print(test)

Wir, die Partei, vom Partei, lehnen natürlich jegliche Partei links von der Partei ab, und die liberalen Menschen von der Partei auch, dafür sind wir freie Demokraten!


Damit bei Ausführung der remove_substrings_replace (r-s-r)-Funktion nicht auch die Begriffe "Europäische Union" und "Grüne Woche" verschwinden, schreiben wir noch eine weitere Funktion, die diese zunächst in andere, vor der r-s-r-Funktion "geschützte" Begriffe umwandelt. Nach Anwendung der r-s-r-Funktion werden diese Begriffe dann wieder in ihren Originalzustand zurückgeführt.

In [ ]:
def change_multiple_strings(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

change_these  = {"Europäischen Union": "fill_word_1", "Europäische Union": "fill_word_2",
                  "europäischen Union": "fill_word_3", "europäische Union": "fill_word_4",
                  "Grünen Woche" : "fill_word_5", "Grüne Woche" : "fill_word_6"}

reverse_these  = {"fill_word_1": "Europäischen Union", "fill_word_2": "Europäische Union",
                  "fill_word_3": "europäischen Union", "fill_word_4": "europäische Union",
                  "fill_word_5": "Grünen Woche",  "fill_word_6": "Grüne Woche"}

Testung der Funktion an einem Testobjekt:

In [ ]:
my_sentence = "Wir finden, man sollte der Europäischen Union ganz schnell zu einer Grünen Woche verhelfen! Die Grüne Woche ist einfach top. Da kann sich die Union, egal ob europäische Union oder nicht, doch etwas abschauen!"
sentence2 = change_multiple_strings(my_sentence, change_these)
print(sentence2)
my_sentence = change_multiple_strings(sentence2, reverse_these)
print(my_sentence)


Wir finden, man sollte der fill_word_1 ganz schnell zu einer fill_word_5 verhelfen! Die fill_word_6 ist einfach top. Da kann sich die Union, egal ob fill_word_4 oder nicht, doch etwas abschauen!
Wir finden, man sollte der Europäischen Union ganz schnell zu einer Grünen Woche verhelfen! Die Grüne Woche ist einfach top. Da kann sich die Union, egal ob europäische Union oder nicht, doch etwas abschauen!


Erste Funktion zum "Schutz" der EU ;):

In [ ]:
df_snippets_noparty = df_snippets.copy()
df_snippets_noparty['Rede_bereinigt'] = df_snippets_noparty['Rede_bereinigt'].apply(lambda s: change_multiple_strings(s, change_these))

Haupt-Funktion auf den Datensatz anwenden:

In [ ]:
df_snippets_noparty['Rede_bereinigt'] = df_snippets_noparty['Rede_bereinigt'].apply(lambda s: remove_substrings_replace(s, substrings = substrings, replacement = "Partei"))

Zurück zur EU:

In [ ]:
df_snippets_noparty['Rede_bereinigt'] = df_snippets_noparty['Rede_bereinigt'].apply(lambda s: change_multiple_strings(s, reverse_these))

Wir überprüfen, ob das geklappt hat:


In [ ]:
print(df_snippets.iloc[30]['Rede_bereinigt'])
print(df_snippets_noparty.iloc[30]['Rede_bereinigt'])

print(df_snippets.iloc[31]['Rede_bereinigt'])  #"Liebe Zuhörerinnen...." war wohl das Ende einer langen Anrede.
print(df_snippets_noparty.iloc[31]['Rede_bereinigt'])

Wir debattieren heute einen Gesetzentwurf des Bundesrates, der vorsieht, einen neuen Paragrafen in das StGB, 90c, einzuführen und damit die Verunglimpfung der Symbole und auch der Flagge der Europäischen Union unter Strafe zu stellen. Ich will vorausschicken, dass bislang die Symbole des Bundes und der Länder geschützt sind, weil wir unsere eigene Verfasstheit, den eigenen Staat und die den Staat ausmachenden Werte besonders schützen wollen und müssen. Ich kann zumindest staatsrechtlich jedem beipflichten, der sagt, dass die Europäische Union kein Staat ist. Sie ist auch kein Bundesstaat. Aber sie ist mittlerweile mehr als nur irgendeine internationale Organisation. Sie ist ein Staatengebilde sui generis, welches mittelbare Hoheitsgewalt ausübt, welches wie kein anderes internationales Organisationsformat für Frieden, Freiheit, Sicherheit und Grundwerte steht. Aber es geht noch weiter. Ihre Flagge hängt nicht umsonst auch hier im Plenarsaal des Deutschen Bundestages. Sie hängt deswegen

Zum Schluss benennen wir, nachdem alle Vorverarbeitungsschritte erfolgt sind, die Spalte 'Rede_bereinigt' wieder in 'Rede' um (für beide Datensätze):

In [ ]:
df_snippets = df_snippets.drop('Rede', axis = 1)
df_snippets.rename(columns={"Rede_bereinigt": "Rede"}, inplace=True)
df_snippets.head()

,Wahlperiode,Datum,Redner*in-ID,Redner*in Vorname,Redner*in Nachname,Rolle,Fraktion,labels,AnzahlZeichen,AnzahlWoerter,Rede,AnzahlZeichen_b,AnzahlWoerter_b,AnzahlZeichen_s,AnzahlWoerter_s
0,19,15.01.2020,11004699,Astrid,Damerow,keine,CDU/CSU,0,4495,638,Ich habe das in meiner Rede im Mai hier bereit...,4400,626,1346,200
0,19,15.01.2020,11004699,Astrid,Damerow,keine,CDU/CSU,0,4495,638,Die Debatte hat es gezeigt: Niemand hier im Ha...,4400,626,3000,418
1,19,15.01.2020,11004393,Johann,Saathoff,keine,SPD,1,3194,470,In diesen Tagen geht es vor allen Dingen um We...,3090,457,2932,432
2,19,15.01.2020,11003706,Artur,Auernhammer,keine,CDU/CSU,0,3206,458,"Aber ich merke immer mehr, dass diese Internat...",3134,449,2995,428
3,19,15.01.2020,11003604,Friedrich,Ostendorff,keine,Die Grünen,2,3388,491,"Die Studie zeigt sehr wohl, wie die Situation ...",3311,481,2776,409


In [ ]:
df_snippets_noparty = df_snippets_noparty.drop('Rede', axis = 1)
df_snippets_noparty.rename(columns={"Rede_bereinigt": "Rede"}, inplace=True)
df_snippets_noparty.head()

,Wahlperiode,Datum,Redner*in-ID,Redner*in Vorname,Redner*in Nachname,Rolle,Fraktion,labels,AnzahlZeichen,AnzahlWoerter,Rede,AnzahlZeichen_b,AnzahlWoerter_b,AnzahlZeichen_s,AnzahlWoerter_s
0,19,15.01.2020,11004699,Astrid,Damerow,keine,CDU/CSU,0,4495,638,Ich habe das in meiner Rede im Mai hier bereit...,4400,626,1346,200
0,19,15.01.2020,11004699,Astrid,Damerow,keine,CDU/CSU,0,4495,638,Die Debatte hat es gezeigt: Niemand hier im Ha...,4400,626,3000,418
1,19,15.01.2020,11004393,Johann,Saathoff,keine,SPD,1,3194,470,In diesen Tagen geht es vor allen Dingen um We...,3090,457,2932,432
2,19,15.01.2020,11003706,Artur,Auernhammer,keine,CDU/CSU,0,3206,458,"Aber ich merke immer mehr, dass diese Internat...",3134,449,2995,428
3,19,15.01.2020,11003604,Friedrich,Ostendorff,keine,Die Grünen,2,3388,491,"Die Studie zeigt sehr wohl, wie die Situation ...",3311,481,2776,409


Wir speichern beide Datensätze:

In [ ]:
df_snippets.to_parquet(path + 'df_snippets.parquet')
df_snippets_noparty.to_parquet(path + 'df_snippets_noparty.parquet')

## Aufteilen in Trainings-, Validierungs- und Testdaten:

Wir laden die beiden benötigten Datensatz:

In [ ]:
df_snippets = pd.read_parquet(path + 'df_snippets.parquet')
df_snippets_noparty = pd.read_parquet(path + 'df_snippets_noparty.parquet')

Wir entfernen unnötige Spalten:

In [ ]:
df_snippets = df_snippets[['Datum', 'Wahlperiode', 'Fraktion', 'labels', 'Rede']]

Wir teilen die Daten entsprechend der Jahre auf:

In [ ]:
df_test = df_snippets[df_snippets['Datum'].str.contains('2024|2025')]
df_val = df_snippets[df_snippets['Datum'].str.contains('2023')]
df_train = df_snippets[df_snippets['Datum'].str.contains('2020|2021|2022')]

#und hier noch zwei separate Trainingsdatensätze für ein "kleineres", sequentielles Training:
df_train2 = df_snippets[df_snippets['Datum'].str.contains('2022')]
df_train3 = df_snippets[df_snippets['Datum'].str.contains('2023')]

Wie lang sind die Datensätze jeweils?

In [ ]:
print(df_test.shape)
print(df_val.shape)
print(df_train.shape)

print(df_train2.shape)
print(df_train3.shape)


(7834, 5)
(8822, 5)
(22426, 5)
(8947, 5)
(8822, 5)


### Balancierung:

Erstmal schauen wir uns an, wie unbalanciert unsere Trainingsdaten eigentlich sind:

In [ ]:
print(df_train["Fraktion"].value_counts(), "\n\n\n")

Fraktion
CDU/CSU       6871
SPD           5400
Die Grünen    2891
FDP           2672
AfD           2612
Die Linke     1980
Name: count, dtype: int64 





In [ ]:
print(df_train2["Fraktion"].value_counts(), "\n\n\n")
print(df_train3["Fraktion"].value_counts(), "\n\n\n")

#Achtung! Hier hat die SPD die meisten Reden.

Fraktion
SPD           2397
CDU/CSU       2272
Die Grünen    1480
FDP           1195
AfD            967
Die Linke      636
Name: count, dtype: int64 



Fraktion
SPD           2447
CDU/CSU       2217
Die Grünen    1436
FDP           1200
AfD            912
Die Linke      610
Name: count, dtype: int64 





Wir definieren eine Funktion, die für die Parteien, die unterrepräsentiert sind, weitere Schnipsel aus den ursprünglichen (noch nicht geteilten) Reden zieht:

In [ ]:
def expand(df_training_cut, df_training_complete, max_zeichen, seed):

  random.seed(seed)

  df_training_filled = df_training_cut
  max_speech = df_training_cut["labels"].value_counts().max()

  for i in range(6):

    num_add = max_speech - (df_training_cut["labels"] == i).sum()

   # aus dem **ursprünglichen** Datensatz df zufällig Reden herauspicken
    added = df_training_complete[df_training_complete["labels"] == i].sample(n = num_add, replace = True, random_state = 42).copy()

    for j in range(len(added)):

      if len(added.iloc[j]["Rede"]) > max_zeichen:

        start = random.choice(range((len(added.iloc[j]["Rede"]) - max_zeichen)))
        end = start + max_zeichen
        rede = added.iloc[j]["Rede"][start:end]

        #Sample, wenn möglich, ganze Sätze (beginne und ende nach diesen)
        start_clean = rede.find(".") + 2
        end_clean = rede.rfind(".") + 1

        added.loc[added.index[j], "Rede"] = rede[start_clean:end_clean]

    df_training_filled = pd.concat([df_training_filled, added])

  df_training_filled.sort_index()
  df_training_filled.reset_index()

  return df_training_filled

Um die Funktion auf unsere Daten anwenden zu können, brauchen wir noch einmal die vollständigen Daten (also vor dem Aufteilen der Reden in kleinere Abschnitte. Diese müssen wir auf die Jahre 2020 bis 2022 einschränken (wie den Trainingsdatensatz). Zusätzlich müssen wir darauf achten, dass alle Datensätze die gleichen Spalten haben.



In [ ]:
df_train_full = df[df['Datum'].str.contains('2020|2021|2022')]
df_train_full = df_train_full.drop('Rede', axis = 1)
df_train_full.rename(columns={"Rede_bereinigt": "Rede"}, inplace=True)
df_train_full = df_train_full[['Datum', 'Wahlperiode', 'Fraktion', 'labels', 'Rede']]

In [ ]:
# fuer die kleineren Trainingsdatensätze hier die vollständigen Daten:

df_train_full2 = df[df['Datum'].str.contains('2022')]
df_train_full2 = df_train_full2.drop('Rede', axis = 1)
df_train_full2.rename(columns={"Rede_bereinigt": "Rede"}, inplace=True)
df_train_full2 = df_train_full2[['Datum', 'Wahlperiode', 'Fraktion', 'labels', 'Rede']]

df_train_full3 = df[df['Datum'].str.contains('2023')]
df_train_full3 = df_train_full3.drop('Rede', axis = 1)
df_train_full3.rename(columns={"Rede_bereinigt": "Rede"}, inplace=True)
df_train_full3 = df_train_full3[['Datum', 'Wahlperiode', 'Fraktion', 'labels', 'Rede']]



Jetzt können wir die Funktion anwenden:

In [ ]:
df_train = expand( df_training_cut = df_train, df_training_complete= df_train_full, max_zeichen = 3000, seed = 42)
df_train2 = expand( df_training_cut = df_train2, df_training_complete= df_train_full2, max_zeichen = 3000, seed = 42)
df_train3 = expand( df_training_cut = df_train3, df_training_complete= df_train_full3, max_zeichen = 3000, seed = 42)

Wir überprüfen, ob es funktioniert hat:

In [ ]:
print(df_train["Fraktion"].value_counts(), "\n\n\n")
print(df_train2["Fraktion"].value_counts(), "\n\n\n")
print(df_train3["Fraktion"].value_counts(), "\n\n\n")

Fraktion
CDU/CSU       6871
SPD           6871
Die Grünen    6871
Die Linke     6871
AfD           6871
FDP           6871
Name: count, dtype: int64 



Fraktion
SPD           2397
CDU/CSU       2397
FDP           2397
AfD           2397
Die Grünen    2397
Die Linke     2397
Name: count, dtype: int64 



Fraktion
SPD           2447
CDU/CSU       2447
FDP           2447
Die Linke     2447
Die Grünen    2447
AfD           2447
Name: count, dtype: int64 





Wir zählen auch noch mal Zeichen und Wörter und schauen uns die Verteilungen pro Partei an:

In [ ]:
df_train['AnzahlZeichen'] = df_train['Rede'].apply(len).astype("int16")
df_train['AnzahlWoerter'] = df_train['Rede'].apply(lambda x: len(x.split())).astype("int16")

df_train2['AnzahlZeichen'] = df_train2['Rede'].apply(len).astype("int16")
df_train2['AnzahlWoerter'] = df_train2['Rede'].apply(lambda x: len(x.split())).astype("int16")

df_train3['AnzahlZeichen'] = df_train3['Rede'].apply(len).astype("int16")
df_train3['AnzahlWoerter'] = df_train3['Rede'].apply(lambda x: len(x.split())).astype("int16")

In [ ]:
print(df_train.groupby('Fraktion')['AnzahlZeichen'].describe())
print(df_train2.groupby('Fraktion')['AnzahlZeichen'].describe())
print(df_train3.groupby('Fraktion')['AnzahlZeichen'].describe())

             count         mean         std     min     25%     50%     75%  \
Fraktion                                                                      
AfD         6871.0  2725.001310  329.698781  1262.0  2698.0  2837.0  2916.0   
CDU/CSU     6871.0  2621.602096  470.807633  1250.0  2446.5  2862.0  2943.0   
Die Grünen  6871.0  2690.830883  359.821709  1245.0  2649.0  2825.0  2913.0   
Die Linke   6871.0  2678.428322  349.189315  1237.0  2614.0  2808.0  2905.0   
FDP         6871.0  2704.295445  348.583080  1270.0  2679.0  2829.0  2913.0   
SPD         6871.0  2677.050502  421.972438  1233.0  2653.5  2859.0  2940.0   

               max  
Fraktion            
AfD         3000.0  
CDU/CSU     3000.0  
Die Grünen  3000.0  
Die Linke   3000.0  
FDP         3000.0  
SPD         3000.0  
             count         mean         std     min     25%     50%     75%  \
Fraktion                                                                      
AfD         2397.0  2701.561535  349.4093

In [ ]:
print(df_train.groupby('Fraktion')['AnzahlWoerter'].describe())
print(df_train2.groupby('Fraktion')['AnzahlWoerter'].describe())
print(df_train3.groupby('Fraktion')['AnzahlWoerter'].describe())

# die Linke sticht etwas heraus

             count        mean        std    min    25%    50%    75%    max
Fraktion                                                                    
AfD         6871.0  384.258769  48.833696  200.0  371.0  396.0  414.0  475.0
CDU/CSU     6871.0  379.207102  69.181949  200.0  351.0  405.0  428.0  483.0
Die Grünen  6871.0  387.277543  53.844146  200.0  373.0  401.0  422.0  483.0
Die Linke   6871.0  382.011061  52.153844  200.0  364.0  395.0  417.0  482.0
FDP         6871.0  388.434435  52.623499  200.0  374.0  402.0  421.0  495.0
SPD         6871.0  385.831466  62.315111  200.0  372.0  406.0  426.0  495.0
             count        mean        std    min    25%    50%    75%    max
Fraktion                                                                    
AfD         2397.0  383.161869  51.351115  200.0  367.0  395.0  415.0  475.0
CDU/CSU     2397.0  382.702128  67.494628  200.0  355.0  408.0  429.0  483.0
Die Grünen  2397.0  382.912390  57.996580  200.0  363.0  400.0  421.0  479.0

Jetzt erstellen wir die Versionen ohne Parteinamen, wobei wir genauso vorgehen wie oben erprobt:

In [ ]:
df_train_np = df_train.copy()
df_val_np = df_val.copy()
df_test_np = df_test.copy()
df_train2_np = df_train2.copy()
df_train3_np = df_train3.copy()

df_train_np['Rede'] = df_train_np['Rede'].apply(lambda s: change_multiple_strings(s, change_these))
df_val_np['Rede'] = df_val_np['Rede'].apply(lambda s: change_multiple_strings(s, change_these))
df_test_np['Rede'] = df_test_np['Rede'].apply(lambda s: change_multiple_strings(s, change_these))
df_train2_np['Rede'] = df_train2_np['Rede'].apply(lambda s: change_multiple_strings(s, change_these))
df_train3_np['Rede'] = df_train3_np['Rede'].apply(lambda s: change_multiple_strings(s, change_these))

df_train_np['Rede'] = df_train_np['Rede'].apply(lambda s: remove_substrings_replace(s, substrings = substrings, replacement = "Partei"))
df_val_np['Rede'] = df_val_np['Rede'].apply(lambda s: remove_substrings_replace(s, substrings = substrings, replacement = "Partei"))
df_test_np['Rede'] = df_test_np['Rede'].apply(lambda s: remove_substrings_replace(s, substrings = substrings, replacement = "Partei"))
df_train2_np['Rede'] = df_train2_np['Rede'].apply(lambda s: remove_substrings_replace(s, substrings = substrings, replacement = "Partei"))
df_train3_np['Rede'] = df_train3_np['Rede'].apply(lambda s: remove_substrings_replace(s, substrings = substrings, replacement = "Partei"))

df_train_np['Rede'] = df_train_np['Rede'].apply(lambda s: change_multiple_strings(s, reverse_these))
df_val_np['Rede'] = df_val_np['Rede'].apply(lambda s: change_multiple_strings(s, reverse_these))
df_test_np['Rede'] = df_test_np['Rede'].apply(lambda s: change_multiple_strings(s, reverse_these))
df_train2_np['Rede'] = df_train2_np['Rede'].apply(lambda s: change_multiple_strings(s, reverse_these))
df_train3_np['Rede'] = df_train3_np['Rede'].apply(lambda s: change_multiple_strings(s, reverse_these))

Zum Schluss speichern wir die Datensätze ab:



In [ ]:
df_train.to_parquet(path + 'df_train.parquet')
df_train2.to_parquet(path + 'df_train2.parquet')
df_train3.to_parquet(path + 'df_train3.parquet')
df_val.to_parquet(path + 'df_val.parquet')
df_test.to_parquet(path + 'df_test.parquet')

df_train_np.to_parquet(path + 'df_train_np.parquet')
df_train2_np.to_parquet(path + 'df_train2_np.parquet')
df_train3_np.to_parquet(path + 'df_train3_np.parquet')
df_val_np.to_parquet(path + 'df_val_np.parquet')
df_test_np.to_parquet(path + 'df_test_np.parquet')